In [9]:
#DON'T RUN THIS CELL IT IS FOR GENERATING PASSWORD IN TO YOUR MAIL !!!
from maticalgos.historical import historical
ma = historical('asec.sem7@gmail.com')
ma.reset_password() 

In [1]:
from maticalgos.historical import historical

maticAlgoHistorical = historical('asec.sem7@gmail.com')
maticAlgoHistorical.login("742986")

In [2]:
import datetime
import sqlite3
import tqdm
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
connection = sqlite3.connect(r"D:\GIT\option-intra\minuteTickData.sqlite")

In [9]:
maticAlgoHistorical.get_data("banknifty", datetime.date(2024, 4, 1)) 

Exception: Unable to fetch data for 2024-04-01

In [9]:
start_date = datetime.date(2021, 1, 1)
end_date = datetime.date(2024, 5, 5)
for day in range((end_date - start_date).days + 1):
    try:
        eachDate = start_date + datetime.timedelta(days=day)
        data = maticAlgoHistorical.get_data("banknifty", eachDate) 
        data.to_sql('bankNiftyWeekly',con=connection, if_exists='append', index=0)
    except:
        print("not able to fetch data for : ", eachDate, " ", datetime.date.fromisoformat(eachDate.isoformat()).strftime("%A"))

not able to fetch data for :  2021-01-01   Friday
not able to fetch data for :  2021-01-02   Saturday
not able to fetch data for :  2021-01-03   Sunday
not able to fetch data for :  2021-01-04   Monday
not able to fetch data for :  2021-01-05   Tuesday
not able to fetch data for :  2021-01-06   Wednesday
not able to fetch data for :  2021-01-07   Thursday
not able to fetch data for :  2021-01-08   Friday
not able to fetch data for :  2021-01-09   Saturday
not able to fetch data for :  2021-01-10   Sunday
not able to fetch data for :  2021-01-11   Monday
not able to fetch data for :  2021-01-12   Tuesday
not able to fetch data for :  2021-01-13   Wednesday
not able to fetch data for :  2021-01-14   Thursday
not able to fetch data for :  2021-01-15   Friday
not able to fetch data for :  2021-01-16   Saturday
not able to fetch data for :  2021-01-17   Sunday
not able to fetch data for :  2021-01-18   Monday


In [8]:
connection.close()

In [3]:
tradingDates = pd.read_csv("tradingDates.csv")

In [3]:
strikePrices = np.arange(start=6000, stop=30000, step=50)

In [5]:
def calculatePerDay(currentDate):
    try:
        print(currentDate)
        sql_query = """select * from niftyWeekly where date = "{}" """.format(currentDate)
        df = pd.read_sql(sql_query, con = connection)
        df['strikePrice'] = df['symbol'].str.slice(start = -7, stop=-2)
        instrument = df[df['symbol'] == "NIFTY"]
        instrument = instrument.reset_index()
        class dayOHLC():
            open930 = 0
            open = 0
            low = 0
            high = 0
            close = 0
        OHLC = dayOHLC()
        dayOpenIndex = np.searchsorted(strikePrices, instrument[instrument['time'] == "09:30:00"]["open"])
        OHLC.open930 = instrument[instrument['time'] == "09:30:00"]["open"].to_list()[0]
        OHLC.open = instrument['open'][0]
        OHLC.low = instrument['low'].min()
        OHLC.high = instrument['high'].max()
        OHLC.close = instrument['close'].iloc[-1]
        if abs(strikePrices[dayOpenIndex+1] - OHLC.open930) > abs(strikePrices[dayOpenIndex] - OHLC.open930):
            if abs(strikePrices[dayOpenIndex] - OHLC.open930) > abs(strikePrices[dayOpenIndex-1] - OHLC.open930):
                tradeOpenStrikePrice = strikePrices[dayOpenIndex-1][0]
            else:
                tradeOpenStrikePrice = strikePrices[dayOpenIndex][0]
        else:
            if abs(strikePrices[dayOpenIndex+1] - OHLC.open930) > abs(strikePrices[dayOpenIndex-1] - OHLC.open930):
                tradeOpenStrikePrice = strikePrices[dayOpenIndex-1][0]
            else:
                tradeOpenStrikePrice = strikePrices[dayOpenIndex+1][0]   
        currentDayGrid = strikePrices[np.searchsorted(strikePrices, OHLC.low)-3 : np.searchsorted(strikePrices, OHLC.high)+3]
        closeSeries = instrument['close'][15:]
        curr_idx = np.where(currentDayGrid == tradeOpenStrikePrice)[0][0]
        crossingsIdx = {15:tradeOpenStrikePrice}
        for eachElement in closeSeries:
            if eachElement > currentDayGrid[curr_idx + 1]:
                curr_idx = curr_idx + 1
                crossingsIdx[closeSeries[closeSeries == eachElement].index[0]] = currentDayGrid[curr_idx]
            elif eachElement < currentDayGrid[curr_idx - 1]:
                curr_idx = curr_idx - 1
                crossingsIdx[closeSeries[closeSeries == eachElement].index[0]] = currentDayGrid[curr_idx]
            else:
                pass
        crossingsdf = instrument.iloc[list(crossingsIdx.keys())]
        crossingsdf['strikePrice'] = list(crossingsIdx.values())
        crossingsdf = crossingsdf.reset_index()
        crossingsdf.insert(loc=len(crossingsdf.columns), column='entryPriceCE', value=0)
        crossingsdf.insert(loc=len(crossingsdf.columns), column='exitPriceCE', value=0)
        crossingsdf.insert(loc=len(crossingsdf.columns), column='gainCE', value=0)
        crossingsdf.insert(loc=len(crossingsdf.columns), column='exitTimeCE', value=0)
        crossingsdf.insert(loc=len(crossingsdf.columns), column='entryPricePE', value=0)
        crossingsdf.insert(loc=len(crossingsdf.columns), column='exitPricePE', value=0)
        crossingsdf.insert(loc=len(crossingsdf.columns), column='gainPE', value=0)
        crossingsdf.insert(loc=len(crossingsdf.columns), column='exitTimePE', value=0)
        for idx, eachRow in enumerate(crossingsdf.itertuples()):
            mask = (df['time'] == eachRow.time) & (df['strikePrice'] == str(eachRow.strikePrice))
            crossingsdf.loc[idx, 'entryPriceCE'] = -df[mask & (df['symbol'].str.contains('CE'))]['open'].to_list()[0]
            crossingsdf.loc[idx, 'entryPricePE'] = -df[mask & (df['symbol'].str.contains('PE'))]['open'].to_list()[0]
            if idx > 0:
                mask = (df['time'] == eachRow.time) & (df['strikePrice'] == str(crossingsdf['strikePrice'][idx-1]))
                if crossingsdf['strikePrice'][idx-1] > crossingsdf['strikePrice'][idx]:
                    crossingsdf.loc[idx-1, 'exitPricePE'] = df[mask & (df['symbol'].str.contains('PE'))]['open'].to_list()[0]
                    crossingsdf.loc[idx-1, 'exitTimePE'] = df[mask & (df['symbol'].str.contains('PE'))]['time'].to_list()[0]
                else:
                    crossingsdf.loc[idx-1, 'exitPriceCE'] = df[mask & (df['symbol'].str.contains('CE'))]['open'].to_list()[0]
                    crossingsdf.loc[idx-1, 'exitTimeCE'] = df[mask & (df['symbol'].str.contains('CE'))]['time'].to_list()[0]
        for idx, eachRow in enumerate(crossingsdf.itertuples()):
            mask = (df['time'] == "15:29:00") & (df['strikePrice'] == str(eachRow.strikePrice))
            if not eachRow.exitTimeCE:
                crossingsdf.loc[idx, 'exitPriceCE'] = df[mask & (df['symbol'].str.contains('CE'))]['open'].to_list()[0]
                crossingsdf.loc[idx, 'exitTimeCE'] = "15:29:00"
            if not eachRow.exitTimePE:
                crossingsdf.loc[idx, 'exitPricePE'] = df[mask & (df['symbol'].str.contains('PE'))]['open'].to_list()[0]
                crossingsdf.loc[idx, 'exitTimePE'] = "15:29:00"
            crossingsdf.loc[idx, 'gainCE'] = crossingsdf.loc[idx, 'entryPriceCE'] + crossingsdf.loc[idx, 'exitPriceCE']
            crossingsdf.loc[idx, 'gainPE'] = crossingsdf.loc[idx, 'entryPricePE'] + crossingsdf.loc[idx, 'exitPricePE']
        
        gainEndofDay = -(crossingsdf['gainCE'].sum() + crossingsdf['gainPE'].sum())*50
        print("""profit on date: "{}" is {}""".format(currentDate, -(crossingsdf['gainCE'].sum() + crossingsdf['gainPE'].sum())*50))
        return crossingsdf
    except:
	    print("some problem with this date: {}".format(currentDate))
tempdf = calculatePerDay(tradingDates['date'][900])

2023-08-14


C:\Users\uie54988\AppData\Local\Temp\ipykernel_11732\2087439555.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crossingsdf['strikePrice'] = list(crossingsIdx.values())


profit on date: "2023-08-14" is 2204.999999999999


In [ ]:
totalDf = pd.DataFrame(columns=tempdf.columns)
for eachDate in tradingDates.itertuples():
    tempdf = calculatePerDay(eachDate.date)
    totalDf = pd.concat([totalDf, tempdf])

In [8]:
totalDf.to_csv("tradesTakenGrid100.csv")

In [13]:
totalDf = pd.DataFrame(columns=perDaydf.columns)

In [15]:
totalDf = pd.concat([totalDf, perDaydf1])

In [16]:
totalDf

,level_0,index,date,time,symbol,open,high,low,close,oi,volume,strikePrice,entryPriceCE,exitPriceCE,gainCE,exitTimeCE,entryPricePE,exitPricePE,gainPE,exitTimePE
0,15,1498,2023-08-14,09:30:00,NIFTY,19288.40,19288.70,19278.05,19279.75,nan,0,19300,-88.30,104.30,16.00,11:26:00,-77.85,23.05,-54.80,15:29:00
1,131,11910,2023-08-14,11:26:00,NIFTY,19349.00,19350.55,19344.80,19350.55,nan,0,19350,-75.35,103.95,28.60,13:49:00,-73.70,34.05,-39.65,15:29:00
2,274,24450,2023-08-14,13:49:00,NIFTY,19392.85,19401.95,19392.30,19401.80,nan,0,19400,-74.25,99.60,25.35,14:49:00,-65.75,50.60,-15.15,15:29:00
3,334,29988,2023-08-14,14:49:00,NIFTY,19447.25,19453.85,19443.65,19453.80,nan,0,19450,-71.00,62.00,-9.00,15:29:00,-67.85,72.40,4.55,15:29:00
0,15,1402,2023-08-16,09:30:00,NIFTY,19365.20,19367.25,19359.10,19359.65,nan,0,19350,-69.15,82.50,13.35,10:44:00,-53.10,17.75,-35.35,15:29:00
1,89,7937,2023-08-16,10:44:00,NIFTY,19398.05,19401.35,19396.35,19400.35,nan,0,19400,-53.90,67.30,13.40,15:00:00,-59.10,31.00,-28.10,15:29:00
2,345,29941,2023-08-16,15:00:00,NIFTY,19431.55,19455.75,19431.55,19455.30,nan,0,19450,-40.05,40.40,0.35,15:29:00,-52.95,52.65,-0.30,15:29:00


In [4]:
df = pd.read_sql("""
            select * from niftyWeekly where date = "2022-04-19"
""", con = connection)

In [5]:
df['strikePrice'] = df['symbol'].str.slice(start = -7, stop=-2)

In [6]:
df.head()

,date,time,symbol,open,high,low,close,oi,volume,strikePrice
0,2022-04-19,09:15:00,NIFTY,17258.95,17270.20,17238.5500,17269.2500,nan,0,NIF
1,2022-04-19,09:15:00,NIFTY-I,17275.00,17298.00,17262.3008,17294.6992,9900050,167303,NIFTY
2,2022-04-19,09:15:00,NIFTY21APR2214350CE,2038.35,2038.35,2038.3500,2038.3500,100,51,14350
3,2022-04-19,09:15:00,NIFTY21APR2214350PE,0.30,0.45,0.3000,0.3000,329650,12101,14350
4,2022-04-19,09:15:00,NIFTY21APR2214400PE,0.40,0.40,0.3500,0.3500,72500,2051,14400


In [7]:
instrument = pd.read_sql("""
            select * from niftyWeekly where date = "{}"
""".format((datetime.date(2022,4,19) - datetime.timedelta(7)).strftime("%Y-%m-%d")), con = connection)
instrument = instrument[instrument['symbol'] == "NIFTY"]
instrument = instrument.reset_index()
instrument.head()

,index,date,time,symbol,open,high,low,close,oi,volume
0,0,2022-04-12,09:15:00,NIFTY,17584.85,17591.80,17537.25,17572.30,nan,0
1,108,2022-04-12,09:16:00,NIFTY,17570.15,17574.30,17540.80,17560.85,nan,0
2,220,2022-04-12,09:17:00,NIFTY,17562.60,17568.25,17553.05,17556.50,nan,0
3,336,2022-04-12,09:18:00,NIFTY,17556.90,17567.20,17552.35,17563.75,nan,0
4,442,2022-04-12,09:19:00,NIFTY,17566.15,17569.70,17560.95,17564.20,nan,0


In [8]:
class dayOHLC():
    open930 = 0
    open = 0
    low = 0
    high = 0
    close = 0

OHLC = dayOHLC()

In [9]:
dayOpenIndex = np.searchsorted(strikePrices, instrument[instrument['time'] == "09:30:00"]["open"])
OHLC.open930 = instrument[instrument['time'] == "09:30:00"]["open"].to_list()[0]
OHLC.open = instrument['open'][0]
OHLC.low = instrument['low'].min()
OHLC.high = instrument['high'].max()
OHLC.close = instrument['close'].iloc[-1]


In [10]:
if abs(strikePrices[dayOpenIndex+1] - OHLC.open930) > abs(strikePrices[dayOpenIndex] - OHLC.open930):
    if abs(strikePrices[dayOpenIndex] - OHLC.open930) > abs(strikePrices[dayOpenIndex-1] - OHLC.open930):
        tradeOpenStrikePrice = strikePrices[dayOpenIndex][0]
    else:
        tradeOpenStrikePrice = strikePrices[dayOpenIndex+1][0]
else:
    if abs(strikePrices[dayOpenIndex+1] - OHLC.open930) > abs(strikePrices[dayOpenIndex-1] - OHLC.open930):
        tradeOpenStrikePrice = strikePrices[dayOpenIndex][0]
    else:
        tradeOpenStrikePrice = strikePrices[dayOpenIndex+2][0]
print("select strikePrice {} for 9.30 open is {}".format(tradeOpenStrikePrice, OHLC.open930))


select strikePrice 17650 for 9.30 open is 17579.95


In [23]:
currentDayGrid = strikePrices[np.searchsorted(strikePrices, OHLC.low)-5 : np.searchsorted(strikePrices, OHLC.high)+5]

In [24]:
closeSeries = instrument['close'][15:]

In [25]:
curr_idx = np.where(currentDayGrid == tradeOpenStrikePrice)[0][0]
print(curr_idx)
crossingsIdx = {15:tradeOpenStrikePrice}
for eachElement in tqdm.tqdm(closeSeries):
    if eachElement > currentDayGrid[curr_idx + 1]:
        curr_idx = curr_idx + 1
        crossingsIdx[closeSeries[closeSeries == eachElement].index[0]] = currentDayGrid[curr_idx+1]
    elif eachElement < currentDayGrid[curr_idx - 1]:
        curr_idx = curr_idx - 1
        crossingsIdx[closeSeries[closeSeries == eachElement].index[0]] = currentDayGrid[curr_idx+1]
    else:
        pass

9


100%|██████████| 360/360 [00:00<00:00, 35950.32it/s]


In [26]:
crossingsdf = instrument.iloc[list(crossingsIdx.keys())]
crossingsdf['strikePrice'] = list(crossingsIdx.values())
crossingsdf = crossingsdf.reset_index()

C:\Users\uie54988\AppData\Local\Temp\ipykernel_18712\2063398836.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crossingsdf['strikePrice'] = list(crossingsIdx.values())


In [27]:
crossingsdf.insert(loc=len(crossingsdf.columns), column='entryPriceCE', value=0)
crossingsdf.insert(loc=len(crossingsdf.columns), column='exitPriceCE', value=0)
crossingsdf.insert(loc=len(crossingsdf.columns), column='gainCE', value=0)
crossingsdf.insert(loc=len(crossingsdf.columns), column='exitTimeCE', value=0)
crossingsdf.insert(loc=len(crossingsdf.columns), column='entryPricePE', value=0)
crossingsdf.insert(loc=len(crossingsdf.columns), column='exitPricePE', value=0)
crossingsdf.insert(loc=len(crossingsdf.columns), column='gainPE', value=0)
crossingsdf.insert(loc=len(crossingsdf.columns), column='exitTimePE', value=0)

In [28]:
crossingsdf

,level_0,index,date,time,symbol,open,high,low,close,oi,volume,strikePrice,entryPriceCE,exitPriceCE,gainCE,exitTimeCE,entryPricePE,exitPricePE,gainPE,exitTimePE
0,15,1639,2022-04-12,09:30:00,NIFTY,17579.95,17581.45,17567.45,17571.00,nan,0,17650,0,0,0,0,0,0,0,0
1,17,1846,2022-04-12,09:32:00,NIFTY,17562.65,17562.75,17547.80,17547.80,nan,0,17600,0,0,0,0,0,0,0,0
2,49,5032,2022-04-12,10:04:00,NIFTY,17516.10,17517.60,17499.95,17499.95,nan,0,17550,0,0,0,0,0,0,0,0
3,269,24832,2022-04-12,13:44:00,NIFTY,17455.95,17456.70,17449.10,17449.75,nan,0,17500,0,0,0,0,0,0,0,0
4,285,26198,2022-04-12,14:00:00,NIFTY,17494.55,17501.55,17494.55,17500.60,nan,0,17550,0,0,0,0,0,0,0,0
5,298,27385,2022-04-12,14:13:00,NIFTY,17541.25,17552.90,17540.55,17551.35,nan,0,17600,0,0,0,0,0,0,0,0


In [29]:
for idx, eachRow in enumerate(crossingsdf.itertuples()):
    mask = (df['time'] == eachRow.time) & (df['strikePrice'] == str(eachRow.strikePrice))
    crossingsdf.loc[idx, 'entryPriceCE'] = -df[mask & (df['symbol'].str.contains('CE'))]['open'].to_list()[0]
    crossingsdf.loc[idx, 'entryPricePE'] = -df[mask & (df['symbol'].str.contains('PE'))]['open'].to_list()[0]
    if idx > 0:
        mask = (df['time'] == eachRow.time) & (df['strikePrice'] == str(crossingsdf['strikePrice'][idx-1]))
        if crossingsdf['strikePrice'][idx-1] > crossingsdf['strikePrice'][idx]:
            crossingsdf.loc[idx-1, 'exitPricePE'] = df[mask & (df['symbol'].str.contains('PE'))]['open'].to_list()[0]
            crossingsdf.loc[idx-1, 'exitTimePE'] = df[mask & (df['symbol'].str.contains('PE'))]['time'].to_list()[0]
        else:
            crossingsdf.loc[idx-1, 'exitPriceCE'] = df[mask & (df['symbol'].str.contains('CE'))]['open'].to_list()[0]
            crossingsdf.loc[idx-1, 'exitTimeCE'] = df[mask & (df['symbol'].str.contains('CE'))]['time'].to_list()[0]
for idx, eachRow in enumerate(crossingsdf.itertuples()):
    mask = (df['time'] == "15:29:00") & (df['strikePrice'] == str(eachRow.strikePrice))
    if not eachRow.exitTimeCE:
        crossingsdf.loc[idx, 'exitPriceCE'] = df[mask & (df['symbol'].str.contains('CE'))]['open'].to_list()[0]
        crossingsdf.loc[idx, 'exitTimeCE'] = "15:29:00"
    if not eachRow.exitTimePE:
        crossingsdf.loc[idx, 'exitPricePE'] = df[mask & (df['symbol'].str.contains('PE'))]['open'].to_list()[0]
        crossingsdf.loc[idx, 'exitTimePE'] = "15:29:00"
    crossingsdf.loc[idx, 'gainCE'] = crossingsdf.loc[idx, 'entryPriceCE'] + crossingsdf.loc[idx, 'exitPriceCE']
    crossingsdf.loc[idx, 'gainPE'] = crossingsdf.loc[idx, 'entryPricePE'] + crossingsdf.loc[idx, 'exitPricePE']

IndexError: list index out of range

In [38]:
-(crossingsdf['gainCE'].sum() + crossingsdf['gainPE'].sum())*50

-55902.50000000001

In [19]:
crossingsdf

,level_0,index,date,time,symbol,open,high,low,close,oi,volume,strikePrice,entryPriceCE,exitPriceCE,gainCE,exitTimeCE,entryPricePE,exitPricePE,gainPE,exitTimePE
0,15,1639,2022-04-12,09:30:00,NIFTY,17579.95,17581.45,17567.45,17571.00,nan,0,17600,-9.65,4.05,-5.60,15:29:00,-390.20,382.80,-7.40,09:32:00
1,17,1846,2022-04-12,09:32:00,NIFTY,17562.65,17562.75,17547.80,17547.80,nan,0,17550,-14.10,4.40,-9.70,15:29:00,-334.25,349.40,15.15,10:04:00
2,49,5032,2022-04-12,10:04:00,NIFTY,17516.10,17517.60,17499.95,17499.95,nan,0,17500,-16.20,5.05,-11.15,15:29:00,-304.95,272.60,-32.35,13:44:00
3,269,24832,2022-04-12,13:44:00,NIFTY,17455.95,17456.70,17449.10,17449.75,nan,0,17450,-18.85,20.00,1.15,14:00:00,-229.15,597.95,368.80,15:29:00
4,285,26198,2022-04-12,14:00:00,NIFTY,17494.55,17501.55,17494.55,17500.60,nan,0,17500,-13.05,15.05,2.00,14:13:00,-260.65,650.50,389.85,15:29:00
5,298,27385,2022-04-12,14:13:00,NIFTY,17541.25,17552.90,17540.55,17551.35,nan,0,17550,-9.45,4.40,-5.05,15:29:00,-287.65,700.00,412.35,15:29:00


In [83]:
def callPutfinder(strikePrice, expiryDate, time, df):
    symbolCE = "NIFTY" + expiryDate + str(strikePrice) + "CE"
    symbolPE = "NIFTY" + expiryDate + str(strikePrice) + "PE"
    priceCE = df[(df["symbol"] == symbolCE) & (df['time'] == time)]["open"]
    pricePE = df[(df["symbol"] == symbolPE) & (df['time'] == time)]["open"]
    return priceCE, pricePE
expiryDate = datetime.datetime.strftime(datetime.date(2023,10,19), "%d%b%y").upper()
time = datetime.time(9,30,0).isoformat()
callPutfinder(19550, expiryDate, time, df)


(1540    35.4
 Name: open, dtype: float64,
 1541    29.95
 Name: open, dtype: float64)

In [72]:
instrument[instrument[]]

1542    15.25
Name: open, dtype: float64